In [2]:
import numpy as np
from scipy.stats import multivariate_normal
import random
np.random.seed(123)
random.seed(123)

In [3]:
def gaussian_e(gaussian_data, init_means, covariance_mat, weights):
    weighted_total = np.array([np.multiply(weights, [multivariate_normal.pdf(gaussian_data[j], mean=init_means[i], \
                                                    cov=covariance_mat[i]) for i in xrange(0, len(init_means))]) for j in \
                                                    range(0,len(gaussian_data))])
    memberships = np.array([w/sum(w) for w in [wts for wts in weighted_total]])
    return weighted_total, memberships

def gaussian_m(gaussian_data, memberships, init_means, K):
    covariance_mat = np.array([sum([memberships[i][j] * np.matmul(np.transpose(np.matrix(gaussian_data[i] - init_means[j])),\
                                    np.matrix(gaussian_data[i] - init_means[j]))/sum(memberships[:,j]) for i in range(0, \
                                    len(gaussian_data))]) for j in range(0,K)])
    # Forming covariance matrix takes the most amount of time
    weights = np.array([sum(np.array(memberships[:,i])) for i in range(0, K)])/len(gaussian_data)
    init_means = np.array([sum([data*prob for data, prob in zip(gaussian_data, memberships[:,i])])/sum(memberships[:,i]) \
                           for i in range(0,K)])
    return covariance_mat, weights, init_means

def gaussian_em(gaussian_data, K):
    memberships = np.random.dirichlet(np.ones(K),size=len(gaussian_data))
    weights = np.random.dirichlet(np.ones(K), size=1).flatten()
    samples = random.sample(range(0, len(gaussian_data)), K)
    init_means = np.array([gaussian_data[samples[i]] for i in range(0, len(samples))])
    covariance_mat = np.array([sum([memberships[i][j] * np.matmul(np.transpose(np.matrix(gaussian_data[i] - init_means[j])),\
                                    np.matrix(gaussian_data[i] - init_means[j]))/sum(memberships[:,j]) for i in range(0, \
                                    len(gaussian_data))]) for j in range(0,K)])
    
    while True:
        prev_means = init_means
        weighted_total, memberships = gaussian_e(gaussian_data, init_means, covariance_mat, weights)
#         print 'e step done'
        covariance_mat, weights, init_means = gaussian_m(gaussian_data, memberships, init_means, K)
#         print 'm done'
        if np.all(np.absolute(prev_means - init_means) < 10**-4):
            break
    return init_means, covariance_mat

In [ ]:
# Run this and pass the data to get back the means and covariances
means, covariances = gaussian_em(gaussian_data, 2)